In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os

from sklearn.metrics import r2_score

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import Lambda
from torch.utils.data import DataLoader


import pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import wandb

In [71]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
if not os.path.exists('./../convolutional/cifar-10-batches-py/train'):
    train = np.empty([0,3073])
    for i in ([1,2,3,4,5]):
        t = unpickle('./../convolutional/cifar-10-batches-py/data_batch_{}'.format(i))
        train = np.vstack((train,np.hstack((t.get(b'data'),np.array(t.get(b'labels')).reshape(10000,1)))))
    t = unpickle('./../convolutional/cifar-10-batches-py/test_batch')
    test = np.hstack((t.get(b'data'),np.array(t.get(b'labels')).reshape(10000,1)))
    with open('./../convolutional/cifar-10-batches-py/train', 'wb') as file:
        pickle.dump(train, file)
    with open('./../convolutional/cifar-10-batches-py/test', 'wb') as file:
        pickle.dump(test, file)
    
else:
    with open('./../convolutional/cifar-10-batches-py/train','rb') as file:
        train = pickle.load(file)
    with open('./../convolutional/cifar-10-batches-py/test','rb') as file:
        test = pickle.load(file)
#test =  pd.read_csv('./MNIST_CSV/mnist_test.csv', header=None)

In [72]:
test

array([[158, 159, 165, ..., 129, 110,   3],
       [235, 231, 232, ..., 191, 199,   8],
       [158, 158, 139, ...,   3,   7,   8],
       ...,
       [ 20,  19,  15, ...,  53,  47,   5],
       [ 25,  15,  23, ...,  81,  80,   1],
       [ 73,  98,  99, ...,  58,  26,   7]])

In [73]:
target_transform = Lambda(lambda y : torch.eye(10)[y])
transform = Lambda(lambda y : torch.reshape(y,(3,32,32)))
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = torch.from_numpy(annotations_file)
        self.img = torch.from_numpy(img_dir)
        self.target_transform = target_transform
        self.transform = transform


    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.img[idx,:]
        label = self.img_labels[idx]
        if self.target_transform:
            label = self.target_transform(label)
        if self.transform:
            image = self.transform(label)
        return image, label
    

train_dataloader = DataLoader(CustomImageDataset(train[:,-1],train[:,:-1],target_transform = target_transform), batch_size=100, shuffle=True)
test_dataloader = DataLoader(CustomImageDataset(test[:,-1],test[:,:-1],target_transform = target_transform), batch_size=100, shuffle=True)


In [74]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1,3,4,1).float()
        self.conv2 = nn.Conv2d(3,9,4,1).float()
        self.conv3 = nn.Conv2d(9,18, 7, 1).float()

        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)

        # First fully connected layer
        self.fc1 = nn.Linear(4608, 128).float()
        self.fc2 = nn.Linear(128, 10).float()
        #self.fc3 = nn.Linear(10,1)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.xavier_uniform_(self.conv3.weight)
      
    def forward(self, input):

        x = F.relu(self.conv1(input))
        x = self.dropout1(F.relu(self.conv2(x)))
        x = torch.flatten(self.dropout2(F.relu(self.conv3(x))), start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x.float()

    def back(self,D):
        i = 0
        with torch.no_grad():
            for name, param in self.named_parameters():
                param.where(D[i].int().bool(), torch.tensor(0.0))
                i += 1
            """
            D_weight = torch.from_numpy(D[0][:-1,:].T.astype(int)).bool()
            D_bias = torch.from_numpy(D[0][-1,:].T.astype(int)).bool()
            self.fc1.bias.where(D_bias, torch.tensor(0.0))
            self.fc1.weight.where(D_weight, torch.tensor(0.0))
            D_weight = torch.from_numpy(D[1][:-1,:].T.astype(int)).bool()
            D_bias = torch.from_numpy(D[1][-1,:].T.astype(int)).bool()
            self.fc2.bias.where(D_bias, torch.tensor(0.0))
            self.fc2.weight.where(D_weight, torch.tensor(0.0))
            D_weight = torch.from_numpy(D[2][:-1,:].T.astype(int)).bool()
            D_bias = torch.from_numpy(D[2][-1,:].T.astype(int)).bool()
            self.fc3.bias.where(D_bias, torch.tensor(0.0))
            self.fc3.weight.where(D_weight, torch.tensor(0.0))
"""

In [17]:

"""for mom in (0.004,0.001, 0.0009,0.0005):
    for bat in (100,500, 1000):
        for lr in (0.01,0.004,0.001, 0.0009,0.0005,0.0001):
            for ep in (10,15,30,50):
                wandb.init(
                    project="my-awesome-project",
                    config={
                            "learning_rate": lr,
                            "architecture": "fully_connected",
                            "dataset": "MNIST",
                            "epochs": ep,
                            "momentum":mom,
                            "batch_size":bat,
                            })"""
bat = 100
ep = 15
lr = 0.004
mom = 0.0009
torch.manual_seed(42)

train_dataloader = DataLoader(CustomImageDataset(train[:,-1],train[:,:-1],target_transform = target_transform), batch_size=100, shuffle=True)
test_dataloader = DataLoader(CustomImageDataset(test[:,-1],test[:,:-1],target_transform = target_transform), batch_size=100, shuffle=True)

model = Net()
criterion = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=mom)
for ep in range(ep):
        for i, data in enumerate(train_dataloader):
            X, Y = data
            optim.zero_grad()
            output = model(X.reshape(bat,1,28,28).to(device).float()).float()
            
            loss = criterion(Y.to(device).float(),output.squeeze())
            loss.backward()
            optim.step()
            #wandb.log({"loss":loss})
            """if i%500 ==0:
                print(loss.float(),output[0].argmax(),Y[0].argmax(),ep)"""
out = []
y = []
for i, data in enumerate(test_dataloader, 0):
    X, Y = data    
    output = model.forward(X.reshape(bat,1,28,28).to(device).float()).float()
    out.extend(output.cpu().detach().float().numpy())
    y.extend(Y)


print(list(np.subtract([i.argmax()for i in out],[i.argmax()for i in y])).count(0)/len(y),mom, lr, bat, ep)
#                wandb.log({"accuracy":list(np.subtract([i.argmax()for i in out],[i.argmax()for i in y])).count(0)/len(y)})
                
"""
PATH = './MNIST.pth'
torch.save(model.state_dict(), PATH)"""

/tmp/ipykernel_4865/558160761.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc2(x))


KeyboardInterrupt: 

In [17]:
def Train(D, train_dataloader, test_dataloader ,model, epoch,i):
    epochs = epoch
    
    criterion = nn.CrossEntropyLoss()
    optim = torch.optim.SGD(model.parameters(), lr=i, momentum=0.0009)

    for ep in range(epochs):
        model.back(D) 
        for i, data in enumerate(train_dataloader, 0):
            X, Y = data
            optim.zero_grad()
            output = model(X.reshape(100,1,28,28).to(device).float()).float()
            loss = criterion(Y.to(device).float(),output.squeeze())
            loss.backward()
            optim.step()
            model.back(D)
            
    w = []
    for name, param in model.named_parameters():
        w.append(param.cpu().detach().numpy())
        
    out = []
    y = []
    for i, data in enumerate(test_dataloader, 0):
        X, Y = data    
        output = model.forward(X.reshape(100,1,28,28).to(device).float()).float()
        out.extend(output.cpu().detach().float().numpy())
        y.extend(Y.cpu())
    non_z = 0
    tot = 0
    for i in D:
        non_z += int(torch.count_nonzero(i))
        tot += torch.numel(i)
    print(tot)
    return(w,list(np.subtract([i.argmax()for i in out],[i.argmax()for i in y])).count(0)/len(y), non_z/tot)

In [18]:
def init(N,model):
    W = []
    D = []
    for name, param in model.named_parameters():
        weight = param.cpu().detach()
        list_null = torch.tensor([random.randrange(1,(torch.numel(param)), 1) for j in range(2*int((torch.numel(param))/3))])
        print(list_null.shape)
        dead = torch.zeros(torch.numel(param))
        dead[list_null] = 1
        weight = weight.reshape(-1)
        weight[list_null] = 0
        weight = weight.reshape(param.shape).to(device)
        dead = dead.reshape(param.shape).to(device)
        W.append(weight)
        D.append(dead)
    
    return W,D

In [39]:
def randomChange(count,D):
    
    size = D.shape
    if torch.count_nonzero(D)<=count:
        count = torch.count_nonzero(D)-1
    wake_up = np.zeros(D.numel())
    print(D.count_nonzero(),count)
    wake_up[random.sample(list(np.where(D.reshape(-1) == 1)[0]), count)] = 1
    dead = D.reshape(-1)
    
    
    
    return torch.where(dead < torch.from_numpy(wake_up) ,dead,0).reshape(size)


In [40]:
def Prune(W,d,count):
    indices = np.indices(W.shape).reshape(W.ndim,-1).T
    w = np.absolute(W).reshape(-1)
    w = np.hstack((w.reshape(w.shape[0],1),indices))
    w = w[w[:,0].argsort()].astype(int)
    
    for i in range(count):
        if i>=w.shape[0]:
            break
        while d[tuple(w[i,1:])] == 1:#to double check if correct
            w = w[1:,:]
            if i>=w.shape[0]:
                break
        if i>=w.shape[0]:
            break
        d[tuple(w[i,1:])] = 1
        W[tuple(w[i,1:])] = 0
    return W, d
        

In [41]:
def SynapseSearch(gamma, nab, N):
    epsilon = 1
    d = 0
    m =3
    gam = gamma
    #setweights
    torch.manual_seed(42)

    model = Net()
    W,D = init(N,model)
    
    train_dataloader = DataLoader(CustomImageDataset(train[:,-1],train[:,:-1],target_transform = target_transform), batch_size=100, shuffle=True)
    test_dataloader = DataLoader(CustomImageDataset(test[:,-1],test[:,:-1],target_transform = target_transform), batch_size=100, shuffle=True)

    Acc = []
    density = []
    W, acc,dens = Train(D, train_dataloader, test_dataloader, model, 0, 0.004)
    print(acc, 1-dens)
    Acc.append(acc)
    density.append(1-dens)
    for i in range(1,int(nab+N/gamma)):
        d = np.random.binomial(size=1, n=1, p= epsilon)
        if d == 1:
            for l in range(m):    
                D[l] = randomChange(int(gam*(torch.numel(D[l]))),D[l])
        else:
            for l in range(m):
                W[l], D[l] = Prune(W[l], D[l],int(gam*(torch.numel(D[l]))))
        W , acc , dens = Train(D, train_dataloader, test_dataloader, model,2, 0.004)
        #wandb.log({"accuracy":acc, "density":dens})

        print(acc, 1-dens)
        Acc.append(acc)
        density.append(1-dens)
        epsilon = max(0,epsilon-1/nab)
        print(epsilon, i, )
    plt.plot(Acc)
    plt.show()
    plt.plot(density)
    return Acc, density


In [42]:
gamma = 0.2
nab = 10
N = 2
"""wandb.init(
                    project="fully-connected",
                    config={
                            "learning_rate": 0.004,
                            "architecture": "fully_connected",
                            "dataset": "MNIST",
                            "momentum":0.0009,
                            "batch_size":100,
                            "gamma":gamma,
                            "nab":nab,
                            "N":N,
                            })"""
Acc, dens = SynapseSearch(gamma,nab, N)
#wandb.finish()


torch.Size([32])
torch.Size([2])
torch.Size([288])
torch.Size([6])
torch.Size([5292])
torch.Size([12])
torch.Size([393216])
torch.Size([84])
torch.Size([852])
torch.Size([6])


/tmp/ipykernel_97965/3580032988.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc2(x))


599690
0.0975 0.5136637262585669
tensor(26) 9
tensor(1) 0
tensor(212) 86
599690
0.7964 0.5140622655038436
0.9 1
tensor(0) tensor(-1)


ValueError: Sample larger than population or is negative

In [ ]:
int(torch.tensor([0.0]))

0